In [7]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import timm
import numpy as np

# 定義我們的資料集
class MyDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# 載入圖像路徑和標籤
def load_data(root):
    image_paths = []
    labels = []
    folders = os.listdir(root)
    for folder in folders:
        folder_path = os.path.join(root, folder)
        for filename in os.listdir(folder_path):
            image_paths.append(os.path.join(folder_path, filename))
            labels.append(0 if 'benign' in folder else 1)
    return image_paths, labels

# 定義Swin Transformer模型
def create_model():
    model = timm.create_model('swin_tiny_patch4_window7_224', num_classes=2)
    return model

# 定義訓練函數
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    preds = []
    true = []
    for images, labels in tqdm(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.cpu().numpy())
        true.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(dataloader.dataset)
    acc = accuracy_score(true, preds)
    return model, epoch_loss, acc

# 定義驗證函數
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    preds = []
    true = []
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(dataloader.dataset)
    acc = accuracy_score(true, preds)
    return epoch_loss, acc, classification_report(true, preds, target_names=['benign', 'malignant'])

# 設定參數
train_dir = '/home/kevinluo/Benign_Malignant_dataclassifier/train'
valid_dir = '/home/kevinluo/Benign_Malignant_dataclassifier/valid'
test_dir = '/home/kevinluo/Benign_Malignant_dataclassifier/test'
n_epochs = 25
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 載入資料
train_image_paths, train_labels = load_data(train_dir)
valid_image_paths, valid_labels = load_data(valid_dir)
all_image_paths = train_image_paths + valid_image_paths
all_labels = train_labels + valid_labels

# 資料增強
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 10折交叉驗證
skf = StratifiedKFold(n_splits=10, shuffle=True)

for fold, (train_index, val_index) in enumerate(skf.split(all_image_paths, all_labels)):
    print(f"Fold {fold+1}")

    train_fold_image_paths = [all_image_paths[i] for i in train_index]
    train_fold_labels = [all_labels[i] for i in train_index]

    val_fold_image_paths = [all_image_paths[i] for i in val_index]
    val_fold_labels = [all_labels[i] for i in val_index]

    train_dataset = MyDataset(train_fold_image_paths, train_fold_labels, train_transform)
    val_dataset = MyDataset(val_fold_image_paths, val_fold_labels, test_transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)

    model = create_model().to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(n_epochs):
        print(f"Epoch {epoch+1}")
        model, train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, report = validate(model, val_loader, criterion, device)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print(report)


# 測試
test_image_paths, test_labels = load_data(test_dir)
test_dataset = MyDataset(test_image_paths, test_labels, test_transform)
test_loader = DataLoader(test_dataset, batch_size=32)
test_loss, test_acc, report = validate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
print(report)


Fold 1
Epoch 1


100%|██████████| 15/15 [00:33<00:00,  2.24s/it]


Train Loss: 0.7975, Train Acc: 0.5618
Val Loss: 0.6863, Val Acc: 0.5388
              precision    recall  f1-score   support

      benign       0.52      0.90      0.66       225
   malignant       0.65      0.18      0.28       226

    accuracy                           0.54       451
   macro avg       0.58      0.54      0.47       451
weighted avg       0.58      0.54      0.47       451

Epoch 2


100%|██████████| 15/15 [00:31<00:00,  2.11s/it]
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Train Loss: 0.6941, Train Acc: 0.5527
Val Loss: 0.8127, Val Acc: 0.5011
              precision    recall  f1-score   support

      benign       0.00      0.00      0.00       225
   malignant       0.50      1.00      0.67       226

    accuracy                           0.50       451
   macro avg       0.25      0.50      0.33       451
weighted avg       0.25      0.50      0.33       451

Epoch 3


...

100%|██████████| 15/15 [00:08<00:00,  1.80it/s]
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Train Loss: 0.6940, Train Acc: 0.4938
Val Loss: 0.6931, Val Acc: 0.5000
              precision    recall  f1-score   support

      benign       0.00      0.00      0.00       225
   malignant       0.50      1.00      0.67       225

    accuracy                           0.50       450
   macro avg       0.25      0.50      0.33       450
weighted avg       0.25      0.50      0.33       450



100%|██████████| 11/11 [00:07<00:00,  1.43it/s]

Test Loss: 0.6931, Test Acc: 0.5078
              precision    recall  f1-score   support

      benign       0.00      0.00      0.00       158
   malignant       0.51      1.00      0.67       163

    accuracy                           0.51       321
   macro avg       0.25      0.50      0.34       321
weighted avg       0.26      0.51      0.34       321




/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, l